# Lecture 22

## 9.3: Module 9 Notebook 3 ##

In [ ]:
# HIDDEN
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in Data 8.


def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plots.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_table(r):
    """
    Generate a table of 1000 x,y data points in standard units
    whose correlation is approximately equal to r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

## Linear regression: defining the line - Copying over useful line functions

In [ ]:
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)


In [ ]:
def slope(t, x, y):
    r = correlation(t, x, y)
    sd_y = np.std(t.column(y))
    sd_x = np.std(t.column(x))
    
    return r*(sd_y/sd_x)

def intercept(t, x, y):
    return np.mean(t.column(y)) - (slope(t, x, y) * np.mean(t.column(x)))

In [ ]:
example = r_table(0.5)
slope(example, 'x', 'y')

## Regression line vs other lines

In [ ]:
def demographics_errors(slope, intercept):
    # Use four convenient points from the original data
    sample = [[14.7, 33995], [19.1, 61454], [50.7, 71183], [59.5, 105918]]
    demographics.scatter('College%', 'Median Income', alpha=0.5)
    xlims = make_array(5, 75)
    # Plot a line with the slope and intercept you specified:
    plots.plot(xlims, slope * xlims + intercept, lw=4)
    # Plot red lines from each of the four points to the line
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=4)

In [ ]:
def fitted_values(t, x, y):
    """Return an array of the regressions estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

In [ ]:
demographics = Table.read_table('district_demographics2016.csv')
demographics.show(5)

In [ ]:
demographics = demographics.drop(
    'State', 'District', 'Percent voting for Clinton')
demographics.show(5)

In [ ]:
demographics.scatter('College%', 'Median Income')

In [ ]:
correlation(demographics, 'College%', 'Median Income')

In [ ]:
regression_slope = slope(demographics, 'College%', 'Median Income')
regression_intercept = intercept(demographics, 'College%', 'Median Income')
regression_slope, regression_intercept

In [ ]:
predicted = fitted_values(demographics, 'College%', 'Median Income')

In [ ]:
demographics = demographics.with_column(
    'Linear Prediction', predicted)
demographics.scatter('College%')

In [ ]:
actual = demographics.column('Median Income')
errors = actual - predicted

In [ ]:
demographics.with_column('Error', errors)

In [ ]:
np.mean(errors)

In [ ]:
np.mean(errors ** 2) ** 0.5

In [ ]:
demographics_errors(regression_slope, regression_intercept)

In [ ]:
# takes any slope, any intercept

demographics_errors(1500, 20000)

In [ ]:
demographics_errors(-1000, 75000)

## Root Mean Square Error ###

In [ ]:
def show_demographics_rmse(slope, intercept):
    demographics_errors(slope, intercept)
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    prediction = slope * x + intercept
    mse = np.mean((y - prediction) ** 2)
    print("Root mean squared error:", round(mse ** 0.5, 2))

In [ ]:
show_demographics_rmse(-1000, 75000)

In [ ]:
show_demographics_rmse(1500, 20000)

In [ ]:
show_demographics_rmse(regression_slope, regression_intercept)

## Numerical Optimization ###

In [ ]:
# If we use the line 'prediction = ax+b' will have an mse that depends on the slope 'a' and the intercept 'b'

# We could define a function lw_mse that takes 
# the slope and intercept as its arguments and returns the corresponding mse.

# If we experiment with different values, we can find a low-error slope and intercept 
# through trial and error, but that would take a while. 

# Fortunately, there is a Python function that does all the trial and error for us.

# The minimize() function can be used to find the 
# *arguments* of a function for which the function returns its minimum value. 

# Python uses a similar trial-and-error approach, 
# following the changes that lead to incrementally lower output values.

#Let's see some examples of how minimize() works

In [ ]:
# Ex. 1. Minimize a quadratic equation
x = np.arange(1, 3, 0.1)
y = (x-2)**2 + 3
Table().with_columns('x', x, 'y', y).plot('x')

In [ ]:
# Let's put the code above in a function
def f(x):
    return ((x-2)**2) + 3

In [ ]:
# Now we can minimize it, call minimize(). Note, it takes a function as an argument
minimize(f)

In [ ]:
# Ex. 2: Minimize a sinusoidal function 
x = np.arange(-1.5, 1.5, 0.05)
y2 = 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 
Table().with_columns('x', x, 'y', y2).plot('x')

In [ ]:
# Let's put the code in a function
def complicated_function(x):
    return 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 

In [ ]:
# Now, let's minimize it
minimize(complicated_function)

## Minimizing RMSE ##

In [ ]:
# Now that we have a sense of how minimize() works, 
# let's get back to attempting to find the 
# "best" slope and intercept values that minimize the rmse

In [ ]:
# working with the demographics dataset still,
# we first define a function that returns RMSE
# when given a slope and intercept value
def demographics_rmse(any_slope, any_intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = any_slope*x + any_intercept
    return (np.mean((y - estimate) ** 2)) ** 0.5

In [ ]:
# Let's test it with a few values of slope and intercept

# test 1
demographics_rmse(1500, 20000)

In [ ]:
# test 2
demographics_rmse(-1000, 75000)

In [ ]:
# now, let's minimize RMSE and 
# obtain the values of slope and intercept 
# at the lowest value of RMSE

minimize(demographics_rmse)

In [ ]:
# Finally, let's confirm that the regression line
# is indeed the "best" fit, i.e., has the lowest RMSE. 
# We check that by comparing its slope and intercept, 
# with those obtained from minimize()

make_array(regression_slope, regression_intercept)

### Nonlinear Regression ###

In [ ]:
#So far, we developed formulas for the slope and intercept 
# of the regression line through a football shaped scatter diagram. 

# It turns out that the slope and intercept 
# of the least squares line have the same formulas 
# as those we developed, regardless of the shape of the scatter plot.

# Let's examine a different dataset, shotput.csv
# In this data, female atheletes throw a shotput, 
# and their furthest throw is recorded.

# We will examine how that distance compares to their strength
# which is measures in how much weight in kgs,
# they can lift in 1 RM.

In [ ]:
shotput = Table.read_table('shotput.csv')
shotput

In [ ]:
shotput.scatter('Weight Lifted')
# Notice that, this is NOT a football shaped scatter plot. 
# In fact, it seems to have a slight non-linear component

In [ ]:
# As before, let's define a function
# to compute RMSE when presented with 
# slope and intercept of a line
def shotput_linear_rmse(any_slope, any_intercept):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = any_slope*x + any_intercept
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
# We will then call minimize()
# to obtain the values of slope and intercept
# for the line with the lowest RMSE
best_line = minimize(shotput_linear_rmse)
best_line

In [ ]:
# Let's compare these with the values of the regression line
# we will use the slope and intercept methods we defined previously

make_array(slope(shotput, 'Weight Lifted', 'Shot Put Distance'), 
           intercept(shotput, 'Weight Lifted', 'Shot Put Distance'))

In [ ]:
# the two arrays are quite similar
# We conclude that:
# No matter what the shape of the scatter plot, 
# there is a unique line that minimizes the mean squared error of estimation. 

# It is called the regression line, and its slope and intercept are given by
# the equations we defined previously

In [ ]:
# We can therefore use the slope and intercept 
# obtained using the minimize function,
# to plot the regression line 

# First, we read "Weight Lifted" and save it in weights. 
# This is our 'x' value
weights = shotput.column(0)

In [ ]:
# Next, we compute the prediction, i.e., y_estimate
# our 'y' is the "Shot Put Distance"

# The line equation is estimated_y = slope * x + intercept

# best_line contains, slope, and intercept in that order

linear_fit = best_line.item(0)*weights + best_line.item(1)

shotput.with_column(
    'Best Line', linear_fit
).scatter(0)

In [ ]:
# Suppose a linear function isn't the best fit for the data?
# What if a quadratic function fit the data better?

# Quadratic functions have the form below

**Quadratic Function**

$$
f(x) ~=~ ax^2 + bx + c
$$
for constants $a$, $b$, and $c$.



In [ ]:
# We can still use the least squares (RMSE)
# to compute the size of errors even with a quadratic equation

# the only thing that changes is that now our functio takes in 
# 3 parameters instead of 2
def shotput_quadratic_rmse(a, b, c):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = a*(x**2) + b*x + c # notice that our estimate is now computed through a quadratic eqn
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
# Minimize will return to us the values for a, b, and c
# that return the least RMSE
best_quad = minimize(shotput_quadratic_rmse)
best_quad

In [ ]:
# suppose x = weight lifted = 100 kg
# Then predicted shot put distance:

(-0.00104)*(100**2) + 0.2827*100 - 1.5318

In [ ]:
# Let's obtained the predicted shot put distances 
# using a quadratic function/predictor
quad_fit = best_quad.item(0)*(weights**2) + best_quad.item(1)*weights + best_quad.item(2)

In [ ]:
# and visualize them
shotput.with_column('Best Quadratic Curve', quad_fit).scatter(0)

In [ ]:
# Notice the curve, especially on the top right of the plot. 
# Is this a better fit for the data compared with the linear predictor?

# We will discuss model diagnostics next